# Project : Recipe Recommendations Based on Weekly Grocery Sales

#### Notebook 2: 
This notebook covers the following : 
(1)  Web Scraping Grocery Store Websites (Super C and Metro) for promotions (effective as of 11.29.2023)
(2)  Cleaning the Dataframe After Web Scraping 


---------------------------------------------------------------------------------------------------------------------------

### 1. Imports

In [13]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

### 2. Create a Function to Web Scrape the Grocery Store Websites

In [3]:
# This function is applicable to Metro website (similar to the Super C one, but slightly different)
# Looking for the following info: Promotion Item, Brand of Item, Promotion Effectivity Date, Reduced Price. URLs are covered in the Section below. 

def grocery_deals(urls):
    brand_list=[]
    product_list=[]
    reduced_price_list=[]
    date_list=[]

    for index,url in enumerate(urls):
        r=requests.get(url)
        if r.status_code !=200:
            print('This URL did not work')
            print(url)
            continue
        
        soup=bs(r.content,'html.parser')
            
        ### PROMOTIONS PER URL
        ###---------------------------------------------------------------------------------------------------------
        
        deals=soup.find_all(class_="default-product-tile tile-product item-addToCart tile-product--effective-date")
        
        for deal in deals:
            
            ### BRAND OF PRODUCT
            try:
                brand=deal.find(class_="head__brand").text #brand of the product
                brand_list.append(brand)
            except AttributeError:
                brand_list.append('NOT AVAILABLE')
                
            ### NAME OF PRODUCT
            try:
                product=deal.find('div',class_="head__title").text #name of the product
                product_list.append(product)
            except AttributeError:
                product_list.append('NOT AVAILABLE')

            ### PROMOTION DATES
            try:
                date=deal.find('div',class_="pricing__until-date").text
                date_list.append(date)
            except AttributeError:
                date_list.append('NOT AVAILABLE')
            
            ### PROMOTION COST
            try:
                # The prices are indicated in two ways: either $1.99 ea. OR 2/$6.99
                prices=deal.find('div',class_="pricing__sale-price promo-price")
                # (1) First price digits : either "$42.99" OR "2 /"
                first=prices.find('span', class_='price-update').text.strip()

                # (2) Check if there is there is abbr
                try:
                    each=prices.find('abbr').text.strip()
                    final_price1=first+" "+each
                    reduced_price_list.append(final_price1)

                # (3) If the above does not work, it will be the unit/cost format
                except AttributeError:    
                    cost=prices.find('span', class_='price-update').find_next('span').text.strip()
                    final_price2=first+cost
                    reduced_price_list.append(final_price2)
            except AttributeError:
                reduced_price_list.append('Check Website for Promotion Price')

    return brand_list,product_list,reduced_price_list,date_list

In [127]:
# This function is applicable to the SuperC website (similar to the Metro one, but slightly different, added the href for Aisle Category)
# Looking for the following info: Promotion Item, Brand of Item, Promotion Effectivity Date, Reduced Price. URLs are covered in the Section below. 

def grocery_deals_2(urls):
    brand_list=[]
    product_list=[]
    reduced_price_list=[]
    date_list=[]
    aisle_category=[]

    for index,url in enumerate(urls):
        r=requests.get(url)
        if r.status_code !=200:
            print('This URL did not work')
            print(url)
            continue
        
        soup=bs(r.content,'html.parser')
            
        ### PROMOTIONS PER URL
        ###---------------------------------------------------------------------------------------------------------
        
        deals=soup.find_all(class_="default-product-tile tile-product item-addToCart tile-product--effective-date")
        
        for deal in deals:
            
            ### BRAND OF PRODUCT
            try:
                brand=deal.find(class_="head__brand").text #brand of the product
                brand_list.append(brand)
            except AttributeError:
                brand_list.append('NOT AVAILABLE')
                
            ### NAME OF PRODUCT
            try:
                product=deal.find('div',class_="head__title").text #name of the product
                product_list.append(product)
            except AttributeError:
                product_list.append('NOT AVAILABLE')

            ### PROMOTION DATES
            try:
                date=deal.find('div',class_="pricing__until-date").text
                date_list.append(date)
            except AttributeError:
                date_list.append('NOT AVAILABLE')
            
            ### PROMOTION COST
            try:
                # The prices are indicated in two ways: either $1.99 ea. OR 2/$6.99
                prices=deal.find('div',class_="pricing__sale-price promo-price")
                # (1) First price digits : either "$42.99" OR "2 /"
                first=prices.find('span', class_='price-update').text.strip()

                # (2) Check if there is there is abbr
                try:
                    each=prices.find('abbr').text.strip()
                    final_price1=first+" "+each
                    reduced_price_list.append(final_price1)

                # (3) If the above does not work, it will be the unit/cost format
                except AttributeError:    
                    cost=prices.find('span', class_='price-update').find_next('span').text.strip()
                    final_price2=first+cost
                    reduced_price_list.append(final_price2)
            except AttributeError:
                reduced_price_list.append('Check Website for Promotion Price')
                
            ### AISLE CATEGORY
            href=deal.find('a', class_='product-details-link').get('href')
            webpage="super.ca"+href
            aisle_category.append(webpage)


    return brand_list,product_list,reduced_price_list,date_list,aisle_category

### 3. Grocery Store Web Scraping

#### 3.1 Grocery Store : Metro
N.B. The Metro Grocery Store website is made in such a way that you may only filter promotion items one 'aisle' category at a time - you cannot filter the promotions several 'aisle' categories at once. As a result, each 'aisle' category has been scraped one at a time. 'Aisle' categories such as 'baby','home','prepared meals', etc were excluded.

In [4]:
### OBTAIN ALL THE URLS 

# 1. Beer & Wine
url_metro_alcohol=[]
for i in range(1,6):
    url_metro_alcohol.append("https://www.metro.ca/en/online-grocery/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000018&fromEcomFlyer=true".format(i))
    
# 2. Beverages
url_metro_beverages=[]
for i in range(1,9):
    url_metro_beverages.append("https://www.metro.ca/en/online-grocery/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000002&fromEcomFlyer=true".format(i))

# 3. Bread & Bakery Products
url_metro_bread=[]
for i in range (1,4):
    url_metro_bread.append("https://www.metro.ca/en/online-grocery/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000012&fromEcomFlyer=true".format(i))
    
# 4. Dairy & Eggs
url_metro_dairy=[]
for i in range(1,8):
    url_metro_dairy.append("https://www.metro.ca/en/online-grocery/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000008&fromEcomFlyer=true".format(i))

# 5. Fish & Seafood
url_metro_fish=[]
for i in range(1,4):
    url_metro_fish.append("https://www.metro.ca/en/online-grocery/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000007&fromEcomFlyer=true".format(i))
    
# 6. Frozen
url_metro_frozen=[]
for i in range(1,6):
    url_metro_frozen.append("https://www.metro.ca/en/online-grocery/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000011&fromEcomFlyer=true".format(i))
    
# 7. Fruits & Vegetables
url_metro_fruits_veggies=[]
for i in range(1,3):
    url_metro_fruits_veggies.append("https://www.metro.ca/en/online-grocery/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000006&fromEcomFlyer=true".format(i))

# 8. Meat & Poultry
url_metro_meat=[]
for i in range(1,5):
    url_metro_meat.append("https://www.metro.ca/en/online-grocery/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000001&fromEcomFlyer=true".format(i))

# 9. Organic Groceries
url_metro_organic=["https://www.metro.ca/en/online-grocery/search?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000017&fromEcomFlyer=true"]

# 10. Pantry
url_metro_pantry=[]
for i in range(1,12):
    url_metro_pantry.append("https://www.metro.ca/en/online-grocery/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000010&fromEcomFlyer=true".format(i))
    
# 11. Snacks
url_metro_snacks=[]
for i in range(1,10):
    url_metro_snacks.append("https://www.metro.ca/en/online-grocery/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000013&fromEcomFlyer=true".format(i))

# 12. Vegan & Vegetarian
url_metro_vegan=["https://www.metro.ca/en/online-grocery/search?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000021&fromEcomFlyer=true"]

# 13. World Cuisine
url_metro_world=["https://www.metro.ca/en/online-grocery/search?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals%3AfirstTopCategory%3A000019&fromEcomFlyer=true"]

##### The Metro Website cannot take too many requests at once --- must only submit a few URLs at a time

In [6]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_alcohol)
metro1 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro1['Aisle']="Beer & Wine"

In [8]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_beverages)
metro2 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro2['Aisle']="Beverages"

In [9]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_bread)
metro3 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro3['Aisle']="Bread & Bakery Products"

In [10]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_dairy)
metro4 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro4["Aisle"]="Dairy & Eggs"

In [11]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_fish)
metro5 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro5['Aisle']="Fish & Seafood"

In [12]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_frozen)
metro6 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro6['Aisle']="Frozen"

In [13]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_fruits_veggies)
metro7 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro7['Aisle']="Fruits & Vegetables"

In [14]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_meat)
metro8 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro8['Aisle']="Meat & Poultry"

In [15]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_organic)
metro9 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro9['Aisle']="Organic Groceries"

In [16]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_pantry)
metro10 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro10['Aisle']="Pantry"

In [17]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_snacks)
metro11 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro11['Aisle']="Snacks"

In [18]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_vegan)
metro12 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro12['Aisle']="Vegan & Vegetarian"

In [19]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(url_metro_world)
metro13 = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
metro13['Aisle']="World Cuisine"

In [20]:
# Combine all the created dataframes
df_metro=pd.concat([metro1,metro2,metro3,metro4,metro5,metro6,metro7,metro8,metro9,metro10,metro11,metro12,metro13], axis=0, ignore_index=True)
df_metro

,Brand,Product,Deal Price,Promotion Effectivity,Aisle
0,\r\n Valle della Rosa\r...,Italian Red Wine,$11.62 ea.,"\nValid until Dec 6, 2023\n",Beer & Wine
1,\r\n Bù\r\n ...,Italian White Wine,$12.97 ea.,"\nValid until Dec 6, 2023\n",Beer & Wine
2,\r\n Bù\r\n ...,"Italian White Wine, Splendido",$12.97 ea.,"\nValid until Dec 6, 2023\n",Beer & Wine
3,\r\n Budweiser\r\n ...,Blonde Lager Beer,$42.99 ea.,"\nValid until Dec 6, 2023\n",Beer & Wine
4,\r\n Coors Light\r\n ...,Light Lager Beer,2 /$62.98,"\nValid until Dec 6, 2023\n",Beer & Wine
...,...,...,...,...,...
1587,\r\n Not Milk\r\n ...,Gluten Free Chocolate Plant-based Beverage,$3.49 ea.,"\nValid until Dec 6, 2023\n",Vegan & Vegetarian
1588,\r\n Old El Paso\r\n ...,Crunchy Taco Shells,$2.99 ea.,"\nValid until Dec 6, 2023\n",World Cuisine
1589,\r\n Old El Paso\r\n ...,Soft Tortilla Bowls,$2.99 ea.,"\nValid until Dec 6, 2023\n",World Cuisine
1590,\r\n Selection\r\n ...,"4.5"" Flour Tortillas, Street Tacos",$3.79 ea.,"\nValid until Dec 6, 2023\n",World Cuisine


In [27]:
df_metro['Grocery Store']="Metro"

In [28]:
# Save a copy as a backup
df_metro.to_csv('metro_11292023.csv', index=False)

#### 3.2 Grocery Store : Super C
N.B. The Super C Grocery Store website is made in such a way that unlike the Metro Website, you may not filter promotion items by category. As a result, it is necessary to complete a preliminary clean-up to filter out items such as Heal & Beauty, Household, Pet Care, etc.

In [22]:
### OBTAIN ALL THE URLS 

urls_superc=[] #33 pages
for i in range(1,34):
    urls_superc.append("https://www.superc.ca/en/search-page-{}?sortOrder=popularity&filter=%3Apopularity%3Adeal%3AFlyer+%26+Deals%2F%3Adeal%3AFlyer+%26+Deals&fromEcomFlyer=true".format(i))

In [23]:
brand_list,product_list,reduced_price_list,date_list=grocery_deals(urls_superc)
superc = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        })
superc

,Brand,Product,Deal Price,Promotion Effectivity
0,\r\n Svelte\r\n ...,Partially Dealcoholized Canadian White Wine,$11.99 ea.,"\nValid until Dec 6, 2023\n"
1,\n,English Cucumber,$0.99 ea.,"\nValid until Dec 6, 2023\n"
2,\n,Red Cluster Tomatoes,$4.25 avg.,"\nValid until Dec 6, 2023\n"
3,\n,Raspberries,2 /$3.88,"\nValid until Dec 6, 2023\n"
4,\r\n Irresistibles\r\n ...,Frozen Three-Meat Pie,$6.99 ea.,"\nValid until Dec 6, 2023\n"
...,...,...,...,...
993,\r\n AXE\r\n ...,Apollo 2 In 1 Shampoo and Conditioner,$3.85 ea.,"\nValid until Dec 6, 2023\n"
994,\r\n AXE\r\n ...,"Total Fresh 3 In 1 Shampoo, Conditioner and Bo...",$3.85 ea.,"\nValid until Dec 6, 2023\n"
995,\r\n Tradition\r\n ...,Sparkling Apple Juice,$4.99 ea.,"\nValid until Dec 6, 2023\n"
996,\r\n AXE\r\n ...,Ice Chillin' Shower Gel,$3.85 ea.,"\nValid until Dec 6, 2023\n"


In [130]:
brand_list,product_list,reduced_price_list,date_list,aisle_category=grocery_deals_2(urls_superc)
superc = pd.DataFrame({
        'Brand': brand_list,
        'Product': product_list,
        'Deal Price': reduced_price_list,
        'Promotion Effectivity': date_list,
        'Aisle':aisle_category
        })
superc

,Brand,Product,Deal Price,Promotion Effectivity,Aisle
0,\r\n Svelte\r\n ...,Partially Dealcoholized Canadian White Wine,$11.99 ea.,"\nValid until Dec 6, 2023\n",super.ca/en/aisles/beer-wine/wines-cocktails-c...
1,\n,English Cucumber,$0.99 ea.,"\nValid until Dec 6, 2023\n",super.ca/en/aisles/fruits-vegetables/vegetable...
2,\n,Red Cluster Tomatoes,$4.25 avg.,"\nValid until Dec 6, 2023\n",super.ca/en/aisles/fruits-vegetables/vegetable...
3,\n,Raspberries,2 /$3.88,"\nValid until Dec 6, 2023\n",super.ca/en/aisles/fruits-vegetables/fruits/be...
4,\r\n Stella Artois\r\n ...,Belgian Blonde Lager Beer,$29.95 ea.,"\nValid until Dec 6, 2023\n",super.ca/en/aisles/beer-wine/beer-cider/classi...
...,...,...,...,...,...
1017,\r\n Doritos\r\n ...,"Rolled Cheese Flavoured Tortilla Chips, Dinami...",3 /$10.00,"\nValid until Dec 6, 2023\n",super.ca/en/aisles/snacks/salty-snacks/corn-to...
1018,\r\n AXE\r\n ...,Anarchy® Bodyspray,$3.85 ea.,"\nValid until Dec 6, 2023\n",super.ca/en/aisles/health-beauty/men-s-product...
1019,\r\n AXE\r\n ...,"Apollo® Deodorant Stick, Fresh",$3.85 ea.,"\nValid until Dec 6, 2023\n",super.ca/en/aisles/health-beauty/men-s-product...
1020,\r\n Doritos\r\n ...,"Rolled Cheese Flavoured Tortilla Chips, Dinami...",3 /$10.00,"\nValid until Dec 6, 2023\n",super.ca/en/aisles/snacks/salty-snacks/corn-to...


In [132]:
superc["Grocery Store"]="Super C"

In [151]:
# Need to Extract the Aisle Category
from urllib.parse import urlparse

def aisle_category(url):
    parsed_url = urlparse(url)
    path_segments = parsed_url.path.split('/')
    return path_segments[3]

superc['Aisle'] = superc['Aisle'].apply(aisle_category)
superc

In [156]:
# Save a copy as a backup
superc.to_csv('superc_11292023.csv', index=False)

### 3.3 Merging the Two Dataframes & Cleaning Them

In [157]:
df=pd.concat([df_metro,superc], axis=0, ignore_index=True)

In [158]:
# Save a copy as a backup
df.to_csv('grocerypromo_11292023.csv', index=False)

In [14]:
df=pd.read_csv("grocerypromo_11292023.csv")

In [15]:
# 1. Cleaning the 'Brand' Column
df['Brand']= df['Brand'].str.replace('\n', '').str.replace('\r', '')
df['Brand']=df['Brand'].str.lstrip().str.rstrip()

df['Brand'].replace('',"The Item's Brand is Not Available", inplace=True)
#df['Brand'].value_counts()

In [16]:
# 2. Cleaning the 'Promotion Effectivity' Column
df['Promotion Effectivity']=df['Promotion Effectivity'].str.replace('\n', '')
#df['Promotion Effectivity'].value_counts()

In [17]:
# 3. Super C Products that are not actual food
#superc['Aisle'].value_counts() #categories like : health-beauty,household-cleaning,pet-care
categories_to_remove=["health-beauty","household-cleaning","pet-care","value-pack","cooked-meals"]
df=df[df['Aisle'].str.contains('|'.join(categories_to_remove), case=False) == False]
len(df) #2614 to 2504

2504

In [18]:
# 4. Change the Aisle Column Values to Match Metro/SuperC
df['Aisle'].value_counts()
df['Aisle']=df['Aisle'].replace('snacks', "Snacks")
df['Aisle']=df['Aisle'].replace('pantry', "Pantry")
df['Aisle']=df['Aisle'].replace('frozen', "Frozen")
df['Aisle']=df['Aisle'].replace('meat-poultry', "Meat & Poultry")
df['Aisle']=df['Aisle'].replace('dairy-eggs', "Dairy & Eggs")
df['Aisle']=df['Aisle'].replace('beer-wine', "Beer & Wine")
df['Aisle']=df['Aisle'].replace('fruits-vegetables', "Fruits & Vegetables")
df['Aisle']=df['Aisle'].replace('bread-bakery-products', "Bread & Bakery Products")
df['Aisle']=df['Aisle'].replace('meat-poultry', "Meat & Poultry")
df['Aisle']=df['Aisle'].replace('deli-prepared-meals', "Deli-Prepared")
df['Aisle']=df['Aisle'].replace('fish-seafood', "Fish & Seafood")
df['Aisle']=df['Aisle'].replace('organic-groceries', "Organic Groceries")
df['Aisle']=df['Aisle'].replace('vegan-vegetarian-food', "Vegan & Vegetarian")
df['Aisle']=df['Aisle'].replace('beverages', "Beverages")

df['Aisle'].value_counts()

Aisle
Pantry                     510
Snacks                     452
Beverages                  322
Dairy & Eggs               303
Frozen                     243
Beer & Wine                176
Meat & Poultry             140
Bread & Bakery Products    110
Fruits & Vegetables        108
Fish & Seafood              86
Deli-Prepared               28
Organic Groceries           15
Vegan & Vegetarian           7
World Cuisine                4
Name: count, dtype: int64

In [19]:
# 5. Check for duplicates on the basis of multiple columns
df=df.drop_duplicates(subset=['Brand', 'Product', 'Deal Price',"Promotion Effectivity","Grocery Store"], keep='first')
len(df) # 2523 to 2454

2454

In [20]:
df_=df.copy()

In [23]:
# 6. Cleaning the 'Product' Column --- creating a new column to compare before/after

### Remove Numbers
df_["Product_Clean"]=df_["Product"].str.replace('\d+', '',regex=True)

###  Remove Capital Letter
df_["Product_Clean"]=df_["Product_Clean"].str.lower()

# ###  Remove Punctuation
# df_.loc[:,"Product_Clean"]=df_["Product_Clean"].str.replace(r'[^\w\s]+', '')

#Remove Different Type of Punctuation/Characters
def punctuation(text):
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return cleaned_text

df_["Product_Clean"]=df_["Product_Clean"].apply(punctuation)

###  Create a function to simplify the 'Products_Clean'
def find_word(x,y):
    word=x
    replacement=y
    df_.loc[df_['Product_Clean'].str.contains(word, case=False), 'Product_Clean']=replacement
    return df_
find_word("yogurt","yogurt") #will replace the entire string by yogurt if it is a yogurt type product
find_word("coffee","coffee")
find_word("tortilla chips","tortilla chips")

###  Remove "bag of" (ex. bag of lemons)
df_['Product_Clean']=df_['Product_Clean'].str.replace("bag of", '')

###  Remove Stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords=(stopwords.words('english'))

###  Created my own list of words that are used often in the product title
other_words=['organic','country','style',"mix","traditional","cooked","di",'au',"de",
             'mini','frozen',"thin",'roast','roasted','diced',"classic","bbq flavoured",
            "value","pack","gluten","free","light","medium","lightly","gourmet","grilled",
             "roasted","small","medium","large","jumbo","canadian","australian","fresh","flavoured",
            "belgianstyle","frozen","argentinian","bin","chilean","italian","irishstyle","mexican",
            "spanish","belgian","assorted","low","calorie","thick","thin","sliced","deepn","delicious","mild",
            "strong","plus","extra","n","simply","disc","plain","montreal","local","tray","barista","ah","le",
             "tassimo","f","bistro","express","regular","readytouse","fatfree","reduced","readytoserve","instant","one",
            "two","three","four","five"]

stopwords.extend(other_words)

df_['Product_Clean']=df_['Product_Clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

###  Lemmatizing
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
lemmatized_string=[]

for row in range(0,len(df_['Product_Clean'])):
    tokens=nltk.word_tokenize(df_['Product_Clean'].iloc[row])
    #print(tokens)
    lemmatized_string.append( ' '.join([lemmatizer.lemmatize(words) for words in tokens]))

df_['Product_Clean']=lemmatized_string

[nltk_data] Downloading package stopwords to C:\Users\Alicia
[nltk_data]     Ionata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
print(df_.iloc[1000:1010]["Product"])
print(df_.iloc[1000:1010]["Product_Clean"])

1021          Ready-to-Use Fat-Free Chicken Broth
1022                           Taco Seasoning Mix
1023                               Hearts of Palm
1024                               Onion Soup Mix
1025                       Reduced Salt Soy Sauce
1026                 Multi-Grain Cereal, Cheerios
1027                 Original Cereal, Mini-Wheats
1028                             Artichoke Hearts
1029    Four Cheese Alfredo Pasta Sauce, Di Parma
1030                         Fajita Seasoning Mix
Name: Product, dtype: object
1021                       chicken broth
1022                      taco seasoning
1023                          heart palm
1024                          onion soup
1025                      salt soy sauce
1026           multigrain cereal cheerio
1027          original cereal miniwheats
1028                     artichoke heart
1029    cheese alfredo pasta sauce parma
1030                    fajita seasoning
Name: Product_Clean, dtype: object


In [25]:
# Save a copy 
df_.to_csv('grocerydeals_v3.csv', index=False)